In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
cred = pd.read_parquet('../data/credit_history.parquet', engine="fastparquet")
xml = pd.read_xml('../data/geographic_data.xml')
xlsx = pd.read_excel('../data/loan_details.xlsx')
jsonl = pd.read_json('../data/financial_ratios.jsonl', lines=True)
demog = pd.read_csv('../data/demographics.csv')
appl = pd.read_csv('../data/application_metadata.csv')

In [33]:
dfs = [cred, xml, xlsx, jsonl, demog, appl]
key_names = ["customer_number", "id", "customer_id", "cust_num", "cust_id", "customer_ref"]

for i in range(len(dfs)):
    dfs[i] = dfs[i].rename(columns={key_names[i]: "customer_id"})

In [34]:
from functools import reduce

merged = reduce(
    lambda left, right: left.merge(right, on="customer_id", how="inner"),
    dfs
)

In [36]:
merged.to_csv('../data/merged_data.csv', index=False)